# 1. Overview

This project provides various methods to solve the 1D Burgers equation:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
$$
where $u(t, x)$ is the velocity field, $\nu$ is the kinematic viscosity, and $t$ and $x$ are time and space variables, respectively. The domain is defined as $t \in [0, T]$ and $x \in [-1, 1]$ with initial conditions $u(0, x) = -\sin(\pi x)$ and Dirichlet boundary conditions $u(-1,t)=u(1,t)=0$. We will only consider kinematic viscosity $\nu = 10^{-2}/\pi$.

The following methods are implemented:
1. Fourier spectral methods (Galerkin and pseudospectral)
2. Chebyshev-Tau spectral method with ABCN scheme
3. Finite difference method with BDF2 and coordinate transformation
4. Physics-informed neural network (PINNs)

The first three methods are inspired by the paper "Spectral and finite difference solutions of the Burgers equation" by Basdevant, C. et al. Comput. Fluids 14, 23 (1986). This paper will be referenced as Basdevant1986 in the following.

The project uses the Julia programming language as I've been wanting to learn it for a while.

Aside from reading this notebook, I encourage you to check out the code in the repository live on `https://github.com/octionary/BurgersJoint`. The code is organized into modules, each corresponding to a different method.

## Analytical solution

We use the convolution form from Cole's transformation like Basdevant1986. We assume initial conditions $u(x,0) = -\sin{(\pi x)}$. The solution is expressed as
$$u(x,t) = \frac{I_1(x,t)}{I_2(x,t)},$$
where
$$I_1(x,t) = -\int_{-\infty}^{+\infty} \sin(\pi(x-\eta))\, f(x-\eta) \exp\left(-\frac{\eta^2}{4 \nu t}\right)\, \mathrm{d}\eta,$$
$$I_2(x,t) = \int_{-\infty}^{+\infty} f(x-\eta) \exp\left(-\frac{\eta^2}{4 \nu t}\right)\, \mathrm{d}\eta,$$
$$f(y) = \exp{\left(-\frac{\cos{(\pi y})}{2\pi\nu}\right)}.$$
A change of variables $\eta = Sz$ with $S = \sqrt{4 \nu t}$ is used so that
$$d\eta = S \,\mathrm{d}z, \quad \exp\left(-\frac{\eta^2}{4\nu t}\right) = \exp(-z^2),$$
yielding
$$I_1(x,t) = -\int_{-\infty}^{+\infty} \sin(\pi(x-Sz)) f(x-Sz) \exp\left(-z^2\right)\, \mathrm{d}\eta,$$
$$I_2(x,t) = \int_{-\infty}^{+\infty} f(x-Sz) \exp\left(-z^2\right)\, \mathrm{d}\eta,$$
Gauss-Hermite quadrature (with $n$ nodes) can now be used to evaluate these integrals:
$$I_1(x,t) \approx \sum_{i=1}^n w_i \sin(\pi(x-Sz_i))\, f(x-Sz_i)$$
$$I_2(x,t) \approx \sum_{i=1}^n w_i f(x-Sz_i)$$
Here, $z_i$ are the roots of the Hermite polynomial are the corresponding weights $w_i$. I use Julia package *FastGaussQuadrature* to provide the $z_i$ and $w_i$.

